In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
import os

In [3]:
iceberg_warehouse = os.environ.get('ICEBERG_WAREHOUSE')
db_user = os.environ.get('DB_USER')
db_pass = os.environ.get('DB_PASS')

configuration = SparkConf()\
    .setAppName('NYT_ArticleSearch')\
    .setMaster('local[4]')\
    .set('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions')\
    .set('spark.sql.catalog.nyt', 'org.apache.iceberg.spark.SparkCatalog')\
    .set('spark.sql.catalog.nyt.type', 'hadoop')\
    .set('spark.sql.catalog.nyt.warehouse', iceberg_warehouse)


spark = SparkSession.builder.config(conf=configuration).getOrCreate()
    

In [4]:
create_table = """
    CREATE TABLE IF NOT EXISTS nyt.db.test ( 
    vendor_id INT , 
    trip_id INT, 
    trip_distance FLOAT, 
    fare_amount FLOAT, 
    store_and_fwd_flag VARCHAR(255) ) 
    USING iceberg PARTITIONED BY (vendor_id) 
    """

print(create_table)

spark.sql(create_table)


    CREATE TABLE IF NOT EXISTS nyt.db.test ( 
    vendor_id INT , 
    trip_id INT, 
    trip_distance FLOAT, 
    fare_amount FLOAT, 
    store_and_fwd_flag VARCHAR(255) ) 
    USING iceberg PARTITIONED BY (vendor_id) 
    


DataFrame[]

In [7]:
from pyspark.sql.types import DoubleType, FloatType, LongType, StructType,StructField, StringType




schema = StructType([
  StructField("vendor_id", LongType(), True),
  StructField("trip_id", LongType(), True),
  StructField("trip_distance", FloatType(), True),
  StructField("fare_amount", DoubleType(), True),
  StructField("store_and_fwd_flag", StringType(), True)
])

df = spark.createDataFrame([(123, 456, 25.0, 50.75, 'X')], schema)
df.writeTo('nyt.db.test').append()

In [8]:
data = spark.sql('SELECT * FROM nyt.db.test')
data.show()

+---------+-------+-------------+-----------+------------------+
|vendor_id|trip_id|trip_distance|fare_amount|store_and_fwd_flag|
+---------+-------+-------------+-----------+------------------+
|      123|    456|         25.0|      50.75|                 X|
+---------+-------+-------------+-----------+------------------+

